## 1.使用function_call模式

In [3]:

import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_experimental.utilities.python import PythonREPL
from langchain_core.tools import Tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.agents import create_tool_calling_agent, AgentExecutor, create_react_agent

# 加载环境变量
dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
os.environ["TAVILY_API_KEY"] = "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"

# 1. 准备工具
search = TavilySearchResults(max_results=3)
search_tool = Tool(
    func=search.run,
    name="search",
    description="用于检索互联网上的信息"
)

python_repl = PythonREPL()
cacl_tool = Tool(
    name="Calculator",
    func=python_repl.run,
    description="用于执行数学计算"
)

tools = [search_tool, cacl_tool]

# 2. 准备模型
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 3. 准备 Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个有用的多功能助手。"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# 4. 创建 Agent
agent = create_tool_calling_agent(llm, tools, prompt)

# 5. 执行
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

# 6. 调用
# 注意：Tavily 搜索国外数据比较强，如果搜索国内特定数据（如某公司股价）可能不如 Google/Bing 准确
result = agent_executor.invoke({"input": "特斯拉当前股价是多少？比去年上涨了百分之多少"})
print(result["output"])

当前 LangChain 版本: 1.2.4


/var/folders/yb/mnp41mqd05q8jv4g26gj3nk80000gn/T/ipykernel_37406/1361237999.py:19: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  search = TavilySearchResults(max_results=3)




> Entering new AgentExecutor chain...

Invoking: `search` with `特斯拉当前股价`


[{'title': 'TSLA股價— 特斯拉圖表 - TradingView', 'url': 'https://tw.tradingview.com/symbols/NASDAQ-TSLA/', 'content': '我們收集了分析師對Tesla, Inc.未來價格的看法：根據他們的說法，TSLA價格的最高預估值為600.00 USD，最低預估為43.00 USD。 查看TSLA圖表，並閱讀更詳細的Tesla, Inc.股票預測：看看分析師們對Tesla, Inc.有什麼看法，並建議您對其股票採取什麼看法行動。\n\nTSLA在2025年12月22日達到了歷史最高價，價格為498.83 USD，而歷史最低價為1.00 USD，達到時間為2010年7月7日。在TSLA圖表上查看更多價格動態。  \n查看其他股票達到它們的最高和最低價格。\n\nTSLA股票的波動率為1.77%，beta係數為1.66。在圖表上追蹤Tesla, Inc.的股價，並查看最波動性股票列表 — Tesla, Inc.是否在其中？\n\n今天，Tesla, Inc.的市值為 \u202a1.49\u202fT\u202c，較上週下降了 −3.51%。\n\n是的，您可以在TradingView上追蹤Tesla, Inc.的財務報表，包括年度和季度報告。\n\nTesla, Inc.將於2026年1月28日發佈下一份收益報告。透過我們的財報日曆追蹤即將發生的活動。\n\n上一季的TSLA收益為每股0.50 USD，而預估值為0.56 USD，結果超出了−10.42%。下一季度的預期收益為每股0.45 USD。查看更多關於Tesla, Inc.收入的詳情。\n\n上一季的Tesla, Inc.收入為\u202a28.09\u202fB\u202c USD， 儘管估計數字為\u202a26.54\u202fB\u202c USD。下個季度的營收預計將達到\u202a24.78\u202fB\u202c USD。\n\nTSLA上一季的淨收入為\u202a1.37\u202fB\u202c USD，而上一季的淨收入為\u202a1.17\u202

## 2.2 使用ReAct模式
举例1:使用PromptTemplate

In [4]:
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_experimental.utilities.python import PythonREPL
from langchain_core.tools import Tool
from langchain_core.prompts import ChatPromptTemplate,PromptTemplate
from langchain_classic.agents import create_react_agent, AgentExecutor

# 加载环境变量
dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
os.environ["TAVILY_API_KEY"] = "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"

# 1. 准备工具
search = TavilySearchResults(max_results=3)
search_tool = Tool(
    func=search.run,
    name="search",
    description="用于检索互联网上的信息"
)

python_repl = PythonREPL()
cacl_tool = Tool(
    name="Calculator",
    func=python_repl.run,
    description="用于执行数学计算"
)

tools = [search_tool, cacl_tool]

# 2. 准备模型
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 3. 准备 Prompt
template = '''Answer the following questions as best you can. You have access to the following
tools:
{tools}
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
Begin!
Question: {input}
Thought:{agent_scratchpad}'''
prompt = PromptTemplate.from_template(
    template=template,
)

# 4. 创建 Agent
agent = create_react_agent(llm, tools, prompt)

# 5. 执行
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

# 6. 调用
# 注意：Tavily 搜索国外数据比较强，如果搜索国内特定数据（如某公司股价）可能不如 Google/Bing 准确
result = agent_executor.invoke({"input": "特斯拉当前股价是多少？比去年上涨了百分之多少"})
print(result["output"])



> Entering new AgentExecutor chain...
我需要查找特斯拉当前的股价以及去年的股价，以便计算出上涨的百分比。  
Action: search  
Action Input: "特斯拉当前股价 2023年10月"  [{'title': 'Tesla 股票交易指南', 'url': 'https://capital.com/zh-hans/learn/market-guides/trade-tesla', 'content': '到 2013 年底，特斯拉股价回落至 10.03 美元，成为市场调整的一个范例。继 Model S 碰撞起火的新闻报道之后，投资者信心也开始下降。\n\n2020 年对特斯拉来说是重要的一年。尽管新冠疫情对股市和汽车行业产生了负面影响，但该公司的股价却加速上涨。6 月 10 日，特斯拉超越 Toyota，成为市值最高的汽车企业。2020 年 12 月，特斯拉被纳入标普 500 指数，极大地鼓舞了投资者的信心和信任。\n\n2021 年公司股价延续震荡走势，涨跌交替，但总体保持上涨趋势。特斯拉于 2021 年 2 月宣布在比特币上投资 15 亿美元的计划受到了投资者的热情支持，股价上涨，但生产延迟的消息导致当月晚些时候公司股价出现大幅回调。2021 年第一季度财报发布后，特斯拉股价在当年 3 月出现反弹，并在 11 月创下历史新高。\n\n特斯拉报告称，2022 年汽车交付数量和收入均有所增长，但其股价却比 2021 年收盘价下跌了 65%，这在很大程度上是由于 8 月份进行的三比一股票拆分造成的。马斯克还于 2022 年收购了 Twitter（现为"X"），这引发了投资者对他的战略重心转移的担忧。\n\n2023 年，特斯拉股价经历了剧烈动荡，1 月至 7 月上涨了 150%，年底又下跌。汽车销量同比下降、半导体芯片短缺和供应链中断导致汽车生产和交付放缓，此外特斯拉因摄像头故障召回近 20 万辆汽车。\n\n受多种原因影响，下降趋势在 2024 年持续。据特斯拉报告称，2024 年第一季度利润较上年同期下降一半以上，盈利未能达到预期。目前，该股仍然处于波动敏感期，因此建议密切关注特斯拉的关键价格驱动因素。\n\n（过往表现并非未来业绩的可靠指标）\n\n## 影响 Tesla 实时股价的因素有哪些？\n

上述举例1也可以如下写

In [ ]:
from langchain_classic import hub
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_experimental.utilities.python import PythonREPL
from langchain_core.tools import Tool
from langchain_core.prompts import ChatPromptTemplate,PromptTemplate
from langchain_classic.agents import create_react_agent, AgentExecutor

# 加载环境变量
dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
os.environ["TAVILY_API_KEY"] = "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"

# 1. 准备工具
search = TavilySearchResults(max_results=3)
search_tool = Tool(
    func=search.run,
    name="search",
    description="用于检索互联网上的信息"
)

python_repl = PythonREPL()
cacl_tool = Tool(
    name="Calculator",
    func=python_repl.run,
    description="用于执行数学计算"
)

tools = [search_tool, cacl_tool]

# 2. 准备模型
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 3. 使用langchain hub的官方ReAct提示词模板

prompt = hub.pull("hwchase17/react")

# 4. 创建 Agent
agent = create_react_agent(llm, tools, prompt)

# 5. 执行
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

# 6. 调用
# 注意：Tavily 搜索国外数据比较强，如果搜索国内特定数据（如某公司股价）可能不如 Google/Bing 准确
result = agent_executor.invoke({"input": "特斯拉当前股价是多少？比去年上涨了百分之多少"})
print(result["output"])

使用chattemplatetemplate模板

In [1]:
import os
import dotenv

from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_experimental.utilities import PythonREPL
from langchain_core.tools import Tool

from langchain.agents import initialize_agent, AgentType


def build_llm():
    api_key = os.getenv("OPENAI_API_KEY")
    base_url = os.getenv("OPENAI_BASE_URL")

    if not api_key:
        raise ValueError("OPENAI_API_KEY 未设置，请在 .env 或环境变量中配置。")

    # 兼容不同版本 ChatOpenAI 的参数名
    try:
        return ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0,
            openai_api_key=api_key,
            base_url=base_url,
        )
    except TypeError:
        return ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0,
            openai_api_key=api_key,
            openai_api_base=base_url,
        )


def build_tools():
    # Tavily 搜索
    search = TavilySearchResults(max_results=3)
    search_func = getattr(search, "invoke", None) or getattr(search, "run", None)
    if search_func is None:
        raise AttributeError("TavilySearchResults 未找到 invoke/run 方法，请检查 langchain-community 版本。")

    search_tool = Tool(
        name="search",
        func=search_func,
        description="用于检索互联网上的实时信息，例如股价、新闻等。"
    )

    # 计算器（PythonREPL）
    python_repl = PythonREPL()
    calc_tool = Tool(
        name="Calculator",
        func=python_repl.run,
        description="用于执行数学计算"
    )

    return [search_tool, calc_tool]


if __name__ == "__main__":
    dotenv.load_dotenv()

    # 建议放到 .env：TAVILY_API_KEY=xxx
    if not os.getenv("TAVILY_API_KEY"):
        os.environ["TAVILY_API_KEY"] = "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"

    tools = build_tools()
    llm = build_llm()

    # ZERO_SHOT_REACT_DESCRIPTION 是最经典、跨版本最稳的 ReAct 入口
    agent_executor = initialize_agent(
        tools=tools,
        llm=llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        handle_parsing_errors=True,
    )

    result = agent_executor.invoke({
        "input": "特斯拉(TSLA)当前股价是多少？相比去年此时上涨了百分之多少"
    })

    print("\n--- Final Output ---")
    # initialize_agent 有的版本返回 {"output": "..."}，也有版本返回字符串
    if isinstance(result, dict) and "output" in result:
        print(result["output"])
    else:
        print(result)


/var/folders/yb/mnp41mqd05q8jv4g26gj3nk80000gn/T/ipykernel_39699/3828557752.py:71: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent_executor = initialize_agent(




> Entering new AgentExecutor chain...
我需要先查找特斯拉(TSLA)当前的股价，然后再查找去年此时的股价，以计算其上涨的百分比。  
Action: search  
Action Input: "特斯拉 TSLA 当前股价"  
Observation: [{'url': 'https://cn.investing.com/equities/tesla-motors', 'content': '470.67\n\nMSFT\n\n-1.36%\n\n亚马逊\n\n242.60\n\nAMZN\n\n-1.57%\n\n## 常见问题\n\n### 特斯拉(TSLA)现在的股价是多少？\n\n特斯拉现在的股价报447.20\n\n### 特斯拉的股票在哪间交易所挂牌交易？\n\n特斯拉的股票在纳斯达克股票交易所挂牌交易。\n\n### 特斯拉的股票代码是什么？\n\n特斯拉的股票代码是“TSLA。”\n\n### 特斯拉的市值是多少？\n\n截至今天，特斯拉的市值是1.48T。\n\n### 特斯拉的每股收益(TTM)是多少？\n\n特斯拉的每股收益(TTM)是1.45。\n\n### 特斯拉的下一财报日是哪天？\n\n特斯拉会在2026年1月27日公布下一份财报。\n\n### 特斯拉有多少名员工？\n\n特斯拉拥有125665名员工。\n\n### 特斯拉(TSLA)当前的行情是什么？\n\n截至2026年1月13日，特斯拉(TSLA)的交易价格报447.20，前一交易日收于448.96，日内波动范围介于443.95至451.81之间，52周波动范围介于214.25至498.83之间。\n\n### 特斯拉(TSLA)的分析师目标价是多少？\n\n特斯拉的12个月平均目标价为USD409.651，最高为USD600，最低为USD130。20名分析师建议买入，而9名则建议卖出，整体评级为中性，具有-8.40% 下行边际的潜力。\n\n### TSLA的盘前价格是多少？\n\nTSLA的最新盘前价格为451.22，盘前成交量为1,290,000.00，较前一交易日下跌2.26，跌幅为0.50%。\n\n### TSLA的盘后价格是多少？\n\nTSLA的最新盘后股价为446.37，较前一交易日下跌-0.83，跌幅为-0.19